In [1]:
import csv
import numpy as np

ROW = 1002
MID = 3
COL = 3
BLK_SIZE = 3

In [2]:
def readcsv():
    A1, A2, A3 = [], [], []
    try:
        with open('H_L2_1.csv', mode='r') as file:
            csv_reader = csv.reader(file)
            header = next(csv_reader, None)  # Read and skip the header line
            if not header:
                print("Error reading header")
                return [], [], []

            for line in csv_reader:
                try:
                    # Read A1, A2, A3 from the 3rd, 5th, and 7th columns respectively
                    A1.append(float(line[2]) / 4.79)
                    A2.append(float(line[4]) / 4.54)
                    A3.append(float(line[6]) / 4.56)
                except ValueError as e:
                    print(f"Invalid data in line: {line}")
                    continue

        return A1, A2, A3

    except FileNotFoundError:
        print("Error opening file")
        return [], [], []

In [3]:
def matrix_strm(A1, A2, A3):
    # Initialize matrices A and AB_sw
    A = np.zeros((ROW, MID))
    AB_sw = np.zeros((ROW, COL))
    
    # Populate the A matrix based on A1, A2, A3
    for i in range(ROW):
        A[i][0] = A1[i]
        A[i][1] = A2[i]
        A[i][2] = A3[i]

    # Simulate the block-based streaming
    A_stream = []

    for ib in range(ROW // BLK_SIZE):
        for jb in range(COL // BLK_SIZE):
            for kb in range(MID // BLK_SIZE):
                # Prepare input data stream (pack data into A_stream)
                for i in range(BLK_SIZE):
                    for j in range(BLK_SIZE):
                        # Simulate axis_pkt with data
                        axis_data = {
                            'data': A[ib * BLK_SIZE + j][kb * BLK_SIZE + i],
                            'last': (i == BLK_SIZE - 1) and (j == BLK_SIZE - 1)
                        }
                        A_stream.append(axis_data)  # Add to stream

    output_data = [data['data'] for data in A_stream]

    # Return the 1D array as output
    return output_data

In [4]:
A1, A2, A3 = readcsv()  # Read data from the CSV
if A1 and A2 and A3:  # Proceed if data was successfully read
    result = matrix_strm(A1, A2, A3)
    print(result)
else:
    print("Error: Could not read data from the CSV.")

Invalid data in line: ['Item', 'V1', 'A1', 'V2', 'A2', 'V3', 'A3']
[0.040338204592901876, 0.029970772442588726, 0.04922546972860125, -0.27202643171806168, -0.27947136563876651, -0.28134361233480176, 0.2394078947368421, 0.23554824561403512, 0.23592105263157898, 0.043125260960334033, 0.056630480167014614, 0.051881002087682676, -0.286431718061674, -0.28788546255506609, -0.28925110132158588, 0.23278508771929829, 0.2317543859649123, 0.22793859649122811, 0.062292275574112731, 0.057937369519832985, 0.068697286012526101, -0.28788546255506609, -0.28997797356828192, -0.28656387665198235, 0.22717105263157897, 0.22214912280701754, 0.22037280701754386, 0.063557411273486422, 0.077104384133611689, 0.069872651356993734, -0.28751101321585903, -0.28665198237885459, -0.28737885462555063, 0.21308114035087722, 0.212265350877193, 0.20519736842105266, 0.092482254697286007, 0.080720250521920667, 0.1032839248434238, -0.28766519823788544, -0.29370044052863437, -0.29447136563876652, 0.20438157894736844, 0.195546

In [5]:
from pynq import allocate
from pynq import Overlay

overlay = Overlay("ComplexMag301.bit")

In [6]:
datasize  = len(result)
print(overlay.ip_dict.keys())

dict_keys(['auto_parkcalc_0', 'pack_stream_to_blk_0', 'unpack_blk_to_stream_0', 'complex_mag_stream_0', 'calculate_statistics_0', 'axi_dma_0', 'axi_dma_1'])


In [7]:
dma = overlay.axi_dma_0
dma_send = overlay.axi_dma_0.sendchannel
dma_recv = overlay.axi_dma_0.recvchannel

InStrm = overlay.pack_stream_to_blk_0
Mat = overlay.auto_parkcalc_0
Mag = overlay.complex_mag_stream_0
OutStrm = overlay.unpack_blk_to_stream_0




dma2 = overlay.axi_dma_1
dma2_send = overlay.axi_dma_1.sendchannel
dma2_recv = overlay.axi_dma_1.recvchannel

Stat = overlay.calculate_statistics_0

In [8]:
InStrm.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  n = Register(n=0)
}

In [9]:
Mat.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  size = Register(size=0)
}

In [10]:
Mag.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  n = Register(n=0)
}

In [11]:
OutStrm.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  n = Register(n=0)
}

In [12]:
InStrm.register_map.n = datasize
Mat.register_map.size = ROW
Mag.register_map.n = ROW
OutStrm.register_map.n = ROW

In [13]:
input_buffer = allocate(shape=(datasize,),dtype=np.float32)
output_buffer = allocate(shape=(ROW,),dtype=np.float32)

In [14]:
for i in range (datasize): input_buffer[i] = result[i]
for i in range(datasize): print(input_buffer[i])

0.0403382
0.0299708
0.0492255
-0.272026
-0.279471
-0.281344
0.239408
0.235548
0.235921
0.0431253
0.0566305
0.051881
-0.286432
-0.287885
-0.289251
0.232785
0.231754
0.227939
0.0622923
0.0579374
0.0686973
-0.287885
-0.289978
-0.286564
0.227171
0.222149
0.220373
0.0635574
0.0771044
0.0698727
-0.287511
-0.286652
-0.287379
0.213081
0.212265
0.205197
0.0924823
0.0807203
0.103284
-0.287665
-0.2937
-0.294471
0.204382
0.195546
0.193327
0.0948768
0.113347
0.106466
-0.297401
-0.298172
-0.301233
0.187436
0.185941
0.180368
0.119664
0.11509
0.120839
-0.301101
-0.302291
-0.299824
0.179417
0.174434
0.17398
0.119359
0.121712
0.121058
-0.301145
-0.29478
-0.299185
0.171443
0.170537
0.165871
0.12158
0.120186
0.124367
-0.289714
-0.294097
-0.284736
0.165689
0.162382
0.161022
0.120666
0.128985
0.12415
-0.289119
-0.282225
-0.284273
0.152958
0.151373
0.146026
0.136958
0.12977
0.147543
-0.281982
-0.282621
-0.283084
0.144349
0.137509
0.135877
0.139136
0.160351
0.150461
-0.285683
-0.285683
-0.287379
0.127993
0.12

0.159618
0.162518
0.16007
-0.287035
-0.29
-0.28499
0.141727
0.135877
0.13359
0.167004
0.162879
0.173572
-0.287161
-0.285804
-0.286681
0.127833
0.125546
0.119789
0.167638
0.18191
0.175386
-0.285511
-0.287077
-0.285282
0.117641
0.108637
0.107128
0.192103
0.183768
0.202026
-0.286681
-0.282547
-0.28499
0.0986278
0.0970749
0.0912687
0.194461
0.206239
0.202978
-0.277098
-0.281461
-0.269812
0.0898062
0.0854207
0.0831806
0.206421
0.205469
0.207689
-0.275929
-0.263069
-0.269248
0.0762797
0.0744053
0.0670463
0.205877
0.210272
0.207917
-0.259896
-0.26263
-0.256534
0.0649449
0.0553458
0.054022
0.217476
0.211632
0.22568
-0.259269
-0.254969
-0.256367
0.0445154
0.0424119
0.0320374
0.218471
0.236645
0.226864
-0.25261
-0.255136
-0.249729
0.0298436
0.0208863
0.0192412
0.244803
0.237829
0.251491
-0.253048
-0.245198
-0.24856
0.0112888
0.00964339
0.00205665
0.246425
0.255439
0.251096
-0.238142
-0.244551
-0.228998
-0.000228524
-0.00818084
-0.0110601
0.258158
0.256031
0.264145
-0.235491
-0.219457
-0.22714
-0

-0.144667
-0.149379
-0.139592
-0.134432
-0.133735
-0.135347
0.280242
0.280308
0.276057
-0.143263
-0.13325
-0.138007
-0.135476
-0.138699
-0.139833
0.275176
0.27348
0.273172
-0.126544
-0.131844
-0.119702
-0.143798
-0.145365
-0.152292
0.270925
0.272797
0.270793
-0.12532
-0.11209
-0.11789
-0.154209
-0.16218
-0.165098
0.274119
0.273084
0.276652
-0.101851
-0.109509
-0.0902982
-0.177121
-0.179169
-0.186313
0.273722
0.277335
0.275947
-0.100401
-0.0794232
-0.0877149
-0.188273
-0.192411
-0.193065
0.275947
0.274119
0.273348
-0.0719474
-0.0780658
-0.0637018
-0.195766
-0.195418
-0.197856
0.269295
0.268414
0.264251
-0.070318
-0.0550482
-0.0621623
-0.198292
-0.202779
-0.204478
0.263018
0.259493
0.258414
-0.0451711
-0.0533268
-0.0337544
-0.209019
-0.212109
-0.220501
0.255396
0.255066
0.253326
-0.0431776
-0.021068
-0.0315789
-0.223612
-0.23144
-0.23476
0.255529
0.254978
0.256718
-0.0102395
-0.0197539
0.000543684
-0.243549
-0.245031
-0.254322
0.255308
0.256542
0.25467
-0.00869912
0.01178
0.00294496
-0.2

-0.283487
-0.283355
-0.286469
0.125284
0.118052
0.115134
0.164211
0.156031
0.1718
-0.286711
-0.287829
-0.287346
0.11056
0.109514
0.106029
0.165903
0.174267
0.172987
-0.287829
-0.284496
-0.286579
0.104939
0.101933
0.0999311
0.174815
0.174267
0.176324
-0.281404
-0.28489
-0.276338
0.0959666
0.0943987
0.0889958
0.174769
0.180163
0.17646
-0.280548
-0.272939
-0.275877
0.087167
0.0799791
0.0788038
0.186104
0.181396
0.196251
-0.270439
-0.272566
-0.270439
0.0706138
0.0683925
0.0614217
0.188207
0.206214
0.197758
-0.271886
-0.269211
-0.27057
0.0590251
0.0507495
0.0481795
0.216681
0.209046
0.226542
-0.269342
-0.270307
-0.268048
0.0405553
0.039119
0.0312777
0.219011
0.23185
0.228612
-0.269452
-0.264408
-0.2675
0.0297954
0.0250919
0.0221294
0.233767
0.231938
0.238612
-0.259934
-0.263509
-0.253048
0.0145497
0.012328
0.000784113
0.234405
0.248172
0.23967
-0.258443
-0.247061
-0.250811
-0.0026572
-0.0154643
-0.0181217
0.259097
0.249075
0.270066
-0.241798
-0.245877
-0.238772
-0.0288372
-0.0305804
-0.0403

In [15]:
print("DMA Source address     :", hex(dma.register_map.MM2S_SA.Source_Address))
print("DMA Destination address:", hex(dma.register_map.S2MM_DA.Destination_Address))
print("---")
print("Input buffer address   :", hex(input_buffer.physical_address))
print("Output buffer address  :", hex(output_buffer.physical_address))

DMA Source address     : 0x0
DMA Destination address: 0x0
---
Input buffer address   : 0x18050000
Output buffer address  : 0x1804b000


In [16]:
CONTROL_REGISTER = 0x0
InStrm.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
Mat.write(CONTROL_REGISTER, 0x81)
Mag.write(CONTROL_REGISTER, 0x81)
OutStrm.write(CONTROL_REGISTER, 0x81)

In [17]:
%%time
dma_send.transfer(input_buffer)
dma_recv.transfer(output_buffer)
dma_send.wait()
dma_recv.wait()

CPU times: user 847 µs, sys: 0 ns, total: 847 µs
Wall time: 893 µs


In [18]:
for i in range(ROW): print(output_buffer[i])

0.364581
0.36663
0.370444
0.371554
0.373888
0.371862
0.371969
0.369809
0.367964
0.363405
0.364919
0.359891
0.364753
0.361814
0.367085
0.364008
0.369221
0.366792
0.37036
0.367416
0.367088
0.366454
0.361643
0.362803
0.355194
0.356731
0.349951
0.348515
0.345249
0.342751
0.345115
0.339914
0.346935
0.342399
0.350753
0.344912
0.354098
0.347911
0.351519
0.349456
0.346828
0.347751
0.341515
0.341937
0.338115
0.33679
0.33819
0.33454
0.340421
0.334865
0.343456
0.339136
0.349697
0.34414
0.352893
0.349534
0.353852
0.352937
0.352118
0.352328
0.351505
0.352285
0.351941
0.352097
0.356387
0.353907
0.364291
0.360051
0.372281
0.368175
0.378245
0.376082
0.380002
0.380533
0.378525
0.380232
0.373819
0.375984
0.368188
0.373322
0.366795
0.372911
0.36995
0.373884
0.373609
0.37696
0.376614
0.378761
0.378652
0.379761
0.377899
0.376712
0.372996
0.372911
0.367319
0.367304
0.360174
0.36284
0.357122
0.360519
0.355951
0.360221
0.357887
0.363705
0.359794
0.364613
0.360772
0.362057
0.356666
0.356309
0.35164
0.349857
0.

In [19]:
%%time
Pv = output_buffer[98:502] - np.mean(output_buffer[98:502])

CPU times: user 1.3 ms, sys: 91 µs, total: 1.39 ms
Wall time: 1.44 ms


In [20]:
for i in range(len(Pv)): print(Pv[i])

0.0037598
0.00715712
0.00258881
0.0068588
0.00452441
0.0103425
0.00643206
0.0112511
0.00741014
0.00869483
0.00330344
0.0029467
-0.00172192
-0.00350517
-0.00863406
-0.00798699
-0.0123932
-0.0118223
-0.0145583
-0.0114932
-0.0136355
-0.010278
-0.0111847
-0.00551972
-0.00719652
-0.00568721
-0.00600672
-0.0045093
-0.00584802
-0.00760135
-0.00699055
-0.00679919
-0.00571939
-0.00552654
-0.002909
-0.00242689
0.00061968
0.00106075
0.00411373
0.00228831
0.00418547
0.00293782
0.00274891
-4.30346e-05
-0.00155857
-0.00329319
-0.00601259
-0.00744203
-0.0082238
-0.0125767
-0.00702086
-0.00961632
-0.00241226
-0.00514391
0.00246972
-0.00106159
0.00437078
0.000876606
0.00239012
0.00091514
-0.00129685
-0.0010477
-0.00622189
-0.00577137
-0.00930721
-0.0104638
-0.0105193
-0.0138213
-0.0103244
-0.0134455
-0.00698853
-0.0119396
-0.00366986
-0.00846291
-0.000985175
-0.00637069
-0.00364763
-0.00399661
-0.00825951
-0.00751778
-0.0121384
-0.0123637
-0.0141752
-0.016514
-0.0136192
-0.0175698
-0.0102319
-0.0153356

In [21]:
del input_buffer, output_buffer

In [24]:
Stat.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  n = Register(n=404)
}

In [23]:
Stat.register_map.n = len(Pv)
Stat.write(CONTROL_REGISTER, 0x81)

Stat_buffer_input = allocate(shape=(len(Pv),),dtype=np.float32)
for i in range (len(Pv)): Stat_buffer_input[i] = Pv[i]
Stat_buffer_output = allocate(shape=(11,),dtype=np.float32)

In [26]:
%%time
dma2_send.transfer(Stat_buffer_input)
dma2_recv.transfer(Stat_buffer_output)
dma2_send.wait()
dma2_recv.wait()

CPU times: user 663 µs, sys: 47 µs, total: 710 µs
Wall time: 755 µs


In [28]:
for i in range(len(Stat_buffer_output)): print(Stat_buffer_output[i])

1.40159e-09
0.011216
0.489026
-0.0850399
0.0559134
0.011216
2.93698
1.25447
3.68436
412.083
0.0508229


In [35]:
%%time
def calc_statistics(data):
    # Number of data points
    n = len(data)

    # Calculate mean
    mean_val = np.mean(data)

    # Calculate standard deviation
    std_dev = np.std(data)

    # Find min and max for peak to peak
    min_val = np.min(data)
    max_val = np.max(data)

    # Calculate RMS
    rms_val = np.sqrt(np.mean(np.square(data)))

    # Calculate absolute mean for shape and impulse factors
    mean_abs = np.mean(np.abs(data))

    # Calculate skewness and kurtosis
    skewness_val = (np.mean((data - mean_val) ** 3)) / (std_dev ** 3)
    kurtosis_val = (np.mean((data - mean_val) ** 4)) / (std_dev ** 4) - 3.0

    # Output results
    results = {
        'mean': mean_val,
        'std_dev': std_dev,
        'skewness': skewness_val,
        'kurtosis': kurtosis_val,
        'peak_to_peak': max_val - min_val,  # peak to peak
        'rms': rms_val,
        'crest_factor': max_val / rms_val,  # crest factor
        'shape_factor': rms_val / mean_abs, # shape factor
        'impulse_factor': max_val / mean_abs,  # impulse factor
        'margin_factor': max_val / (mean_abs ** 2), # margin factor
        'energy': np.sum(np.square(data))  # energy
    }

    return results

statistics = calc_statistics(Stat_buffer_input)

for key, value in statistics.items():
    print(f'{key}: {value}')

mean: 1.4753624988372849e-09
std_dev: 0.011216023936867714
skewness: 0.4890257441170814
kurtosis: -0.08504041035646814
peak_to_peak: 0.05591341853141785
rms: 0.011216023936867714
crest_factor: 2.936983823776245
shape_factor: 1.2544708251953125
impulse_factor: 3.6843605041503906
margin_factor: 412.08209228515625
energy: 0.05082287639379501
CPU times: user 16.7 ms, sys: 0 ns, total: 16.7 ms
Wall time: 15.5 ms
